# Setup
Ho avuto problemi con il mio neo4j locale quindi nel dubbio ho usato quello su docker
```sh
docker run \
    -p 7474:7474 -p 7687:7687 \
    -v $PWD/data:/data -v $PWD/plugins:/plugins \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4JLABS_PLUGINS=\[\"apoc\"\] \
    neo4j:latest
```
E poi connessione su [localhost:7474](localhost:7474)

In [ ]:
! pip install pip install llama-index-graph-stores-neo4j

In [1]:
import nest_asyncio; nest_asyncio.apply()

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "" # Add your OpenAI API key here

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [ ]:
documents = SimpleDirectoryReader('').load_data() # Add the path to the directory containing your documents here

In [6]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

# Note: used to be `Neo4jPGStore`
graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="hackaton",
    url="bolt://localhost:7687",
)

In [13]:
from llama_index.core import PropertyGraphIndex
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI


# create
index = PropertyGraphIndex.from_documents(
    documents,
    llm=OpenAI("gpt-4o"),
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    property_graph_store=graph_store,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████████████████| 1/1 [00:01<00:00,  1.39s/it]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: "MATCH (n:`Chunk`)\nWITH collect(distinct substring(toString(coalesce(n.`doc_id`, '')), 0, 50)) AS `doc_id_values`,\n     collect(distinct substring(toString(coalesce(n.`_node_type`, '')), 0, 50)) AS `_node_type_values`,\n     collect(distinct substring(toString(coalesce(n.`_node_content`, '')), 0, 50)) AS `_node_content_values`,\n     min(size(coalesce(n.`embedding`, []))) AS `embedding_size_min`, max(size(coalesce(n.`embedding`, []))) AS `embedding_size_max`,\n     collect(distinct substring(toString(coalesce(n.`document_id`, '')), 0, 50)) AS `document_id_values`,\n     collect(distinct substring(toStri